In [45]:

import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import undetected_chromedriver as uc
import lxml
import time


In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--ignore-ssl-errors')
# options.add_argument('--disable-images')
# options.add_argument('--disable-javascript')
options.add_argument("--headless")
options.add_argument("--disable-blink-features=AutomationControlled")
prefs = {
    "profile.managed_default_content_settings.images": 2,
    # "profile.managed_default_content_settings.javascript": 1  # Still not advisable
}
options.add_experimental_option("prefs", prefs)
options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.5845.188 Safari/537.36')


In [ ]:
# driver = uc.Chrome(options=options)
driver = webdriver.Chrome(options=options)

driver.get("https://www.nigelfrank.com/microsoft-jobs")
     # Wait for job listings to load
WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="__next"]/div/div[1]/div[2]/div/div[2]')))
soup = BeautifulSoup(driver.page_source, 'lxml')
# print('sp', soup)
driver.quit()
job_list = soup.find_all('div', class_="sc-AykKG sc-fzXfQW BqA-ds")

print("len", len(job_list))

In [ ]:
def nigelfrank_scraper(url):
    driver = webdriver.Chrome(options=options)

    # driver.get("https://www.nigelfrank.com/microsoft-jobs")
    try:
    
        driver.get(url)
            # Wait for job listings to load
        WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="__next"]/div/div[1]/div[2]/div/div[2]')))
        soup = BeautifulSoup(driver.page_source, 'lxml')
        # driver.quit()
        job_list = soup.find_all('div', class_="sc-AykKG sc-fzXfQW BqA-ds")
        print(f"{url} Jobs found", len(job_list))
        job_data = []
        
        for job in job_list:
            link_em = job.find('a',class_="sc-AykKE fwNdOp")
            # print("link_em", link_em)
            job_url = "https://www.nigelfrank.com/" + link_em['href'] if link_em and link_em.get('href') else None
            job_id = job_url.split("/")[-2] if job_url else None
            title_em = job.find('h3', class_="sc-AykKD fECRJb jobTitle")
            title = title_em.text.strip() if title_em else None
            location_em = job.find('p', class_="location")
            location = location_em.text.strip() if location_em else None
            info_list_em = job.find('ul', class_="particulars" )
            info_em = info_list_em.find_all('li')
            salary = role_type = level = None
            # salary = info_em[0].text.strip() if info_em else None
            # role_type = info_em[1].text.strip() if info_em else None
            # level = info_em[2].text.split(":")[-1].strip() if info_em else None
            if info_list_em:
                    info_em = info_list_em.find_all('li')
                    if len(info_em) > 0: salary = info_em[0].get_text(strip=True)
                    if len(info_em) > 1: role_type = info_em[1].get_text(strip=True)
                    if len(info_em) > 2: level = info_em[2].get_text(strip=True).split(":")[-1].strip()

            job_desc_em = job.find("div", class_="jobDescription")
            job_desc_div = job_desc_em.find("div")
            # job_desc_em_br = job_desc_em.find_all("br")
            description = job_desc_div.text if job_desc_div else ''
            print('job_url', job_id)
            data ={
                "title":title,
                "job_id": job_id,
                "job_url":job_url,
                "location":location,
                "salary":salary,
                "role_type":role_type,
                "level":level,
                "description": description
            }
            
            job_data.append(data)
            
        return job_data    
    except Exception as e:
        print(f"Error scarping {url}:{e}")
        return []
    finally:
        driver.quit()

In [ ]:
# urls base on thr page (1-145)

start_urls = [
    f"https://www.nigelfrank.com/microsoft-jobs?newJobs=&keyword=&location=&jobType=both&page={i}&remote=&security=&salaryFrom=&salaryTo=&salaryCurrency=&segment=&product="
    for i in range(1, 148)
]
final_data = []
for url in start_urls:
    try:

        result = nigelfrank_scraper(url)
        final_data.extend(result)
        time.sleep(1)  # wait abit for for the next request
    except Exception as e:
        print("here error:", e)
    finally:
        print("final data len", len(final_data))
        df = pd.DataFrame(final_data)
        df.to_csv("nigelfranck.csv")
        print(f"saved {len(df)} number of jobs")

# https://www.nigelfrank.com/microsoft-jobs?newJobs=&keyword=&location=&jobType=both&page=3&remote=&security=&salaryFrom=&salaryTo=&salaryCurrency=&segment=&product=